# Imports

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px

# Load in Data

In [2]:
pickle_to_load = "../../FantasyData/data-frames/df_wr_college_2000_2023.pkl"

In [3]:
df_pos = pd.read_pickle(pickle_to_load)

### For saving Data Later

In [4]:
save_path = "../../interactive-2.0/WR/draft-capital/"

# Clean Up Data

In [5]:
df_clean = df_pos.copy().dropna(subset=["draft_ovr"])

# Get Draft Capital Success Rate By Round

Grab the best (lower tier is better) tier a player has been in, for each round of the draft:

In [6]:
tier_counts = df_clean.groupby(['draft_round', 'school', 'player_name'])['tier'].min().reset_index()

Then group this data frame by the draft round to get the total number of players in each round

In [8]:
total_players_per_round = tier_counts \
    .groupby(['draft_round', 'school'])['player_name'] \
    .count() \
    .reset_index(name='total_players')

Grab the number of each players in each tier, by their draft round:

In [9]:
players_in_tier = tier_counts.groupby(['draft_round', 'school', 'tier']).size().reset_index(name='players_in_tier')

Merge the dataframes together:

In [10]:
merged_data = pd.merge(players_in_tier, total_players_per_round, on=['draft_round', 'school'])
merged_data['percentage'] = (merged_data['players_in_tier'] / merged_data['total_players']) * 100

Tier percentagers by draft round:

In [11]:
merged_data.head(5)

,draft_round,school,tier,players_in_tier,total_players,percentage
0,1.0,Ala-Birmingham,1,1,1,100.000000
1,1.0,Alabama,1,3,6,50.000000
2,1.0,Alabama,2,1,6,16.666667
3,1.0,Alabama,7,1,6,16.666667
4,1.0,Alabama,8,1,6,16.666667


# Analysis

In [24]:
df_success = merged_data.query("tier <= 4 and total_players >= 5").groupby("school").apply(lambda x: x)

/tmp/ipykernel_11414/4086825510.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_success = merged_data.query("tier <= 4 and total_players >= 5").groupby("school").apply(lambda x: x)


In [25]:
df_success

draft_round       school  tier  players_in_tier  \
school                                                             
Alabama     1            1.0      Alabama     1                3   
            2            1.0      Alabama     2                1   
LSU         20           1.0          LSU     1                4   
Mississippi 98           2.0  Mississippi     1                2   
            99           2.0  Mississippi     3                1   
USC         127          2.0          USC     1                3   
            128          2.0          USC     2                1   
            129          2.0          USC     4                1   

                 total_players  percentage  
school                                      
Alabama     1                6   50.000000  
            2                6   16.666667  
LSU         20               5   80.000000  
Mississippi 98               6   33.333333  
            99               6   16.666667  
USC         127              5   60.000000  
            128              5   20.000000  
            129              5   20.000000

In [28]:
df_success_2 = merged_data.query("tier <= 4 and percentage > 50 and total_players >= 3") \
    .groupby("school") \
    .apply(lambda x: x)

/tmp/ipykernel_11414/1057492819.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x)


In [29]:
df_success_2

draft_round          school  tier  players_in_tier  \
school                                                                   
Clemson        13           1.0         Clemson     2                2   
LSU            20           1.0             LSU     1                4   
Miami (FL)     25           1.0      Miami (FL)     1                3   
South Carolina 123          2.0  South Carolina     1                3   
USC            55           1.0             USC     2                2   
               127          2.0             USC     1                3   

                    total_players  percentage  
school                                         
Clemson        13               3   66.666667  
LSU            20               5   80.000000  
Miami (FL)     25               4   75.000000  
South Carolina 123              3  100.000000  
USC            55               3   66.666667  
               127              5   60.000000

### Bust DF

In [32]:
df_bust = merged_data.query("tier > 4").groupby(['draft_round', 'school']).agg({"percentage": "sum"})

In [34]:
df_bust.query("percentage > 50")

percentage
draft_round school                      
1.0         Arkansas               100.0
            Central Florida        100.0
            Illinois               100.0
            Minnesota              100.0
            Mississippi            100.0
...                                  ...
7.0         Virginia Tech          100.0
            Wake Forest            100.0
            Washington State       100.0
            West Virginia          100.0
            William & Mary         100.0

[161 rows x 1 columns]

# Highlighting

In [ ]:
# Define a function to highlight rows based on a condition 
def highlight_row(row): 
    # Mapping of pick ranges to background colors
    color_map = {
        1.0: 'yellow',
        2.0: 'lightblue',
        3.0: 'lightgreen',
        4.0: 'lightpink',
        5.0: 'lightgrey',
        6.0: 'lightcoral',
        7.0: 'lightsalmon',
    }
    
    # Get the background color for the current row's pick range
    # Default to white if the pick range is not in the map
    bg_color = color_map.get(row['Draft Round'], 'white')
    
    # Return a list of styles for each column in the row
    return [f'background-color: {bg_color}']*len(row)

In [ ]:
df_1 = df_success.style.apply(highlight_row, axis=1)

In [ ]:
df_1

In [ ]:
# with open(f'{save_path}/success-by-round-table.html', 'w') as file:
#     file.write(df_1.to_html())

In [ ]:
df_2 = df_bust.style.apply(highlight_row, axis=1)

In [ ]:
df_2

In [ ]:
# with open(f'{save_path}/busts-by-round-table.html', 'w') as file:
#     file.write(df_2.to_html())

# Plotly Bar Charts

In [ ]:
df_success["Best Season Tier"] = df_success["Best Season Tier"].astype(float)

In [ ]:
fig = px.bar(df_success, x="Draft Round", y="Success Rate",
             color='Best Season Tier', barmode='group',
             hover_data=['Sample Size of Tier', 'All Players in this Round'],
             )

# fig.write_html(f"{save_path}/hit-rate-v-round-alternative.html")
fig.show()

In [ ]:
df_success["Draft Round"] = df_success["Draft Round"].astype(float)

In [ ]:
fig = px.bar(df_success, x="Best Season Tier", y="Success Rate",
             color='Draft Round', barmode='group',
             hover_data=['Sample Size of Tier', 'All Players in this Round'],
             )

# fig.write_html(f"{save_path}/hit-rate-v-tier-by-round-alternative.html")
fig.show()

In [ ]:
fig = px.bar(df_bust, x="Draft Round", y="Chance of Bust")

fig.write_html(f"{save_path}/bust-rate-v-round.html")
fig.show()